In [ ]:
!git clone https://github.com/LINKS-Foundation-CPE/Master-QCC-2022-2023.git

In [ ]:
!pip install pulser==0.17.3
!pip install pybnb
!pip install networkx==2.8.7
!pip install matplotlib==3.5.1

In [ ]:
%cd /content/Master-QCC-2022-2023/

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

from pulser import Pulse, Sequence, Register
from pulser.devices import Chadoq2
from pulser_simulation import SimConfig, Simulation
from pulser.waveforms import InterpolatedWaveform

from scipy.optimize import minimize, Bounds
from src.basic_MIS.utils import *
from scipy.spatial.distance import pdist, squareform

In [ ]:
# pos = np.array([[0., 0.], [-4, -7], [4,-7], [8,6], [-8,6]])
pos = np.array([[0., 0.], [-4, -7], [4,-7], [8,6], [-8,6], [8,16], [-8,16], [0,20]])
# pos = np.array([[0., 0.], [-4, -7], [4,-7], [8,6], [-8,6], [8,16], [-8,16], [0,20], [16,18], [-18,12]])

G = pos_to_graph(pos)
qubits = dict(enumerate(pos))

reg = Register(qubits)
reg.draw(blockade_radius=Chadoq2.rydberg_blockade_radius(1.), draw_graph=True, draw_half_radius=True ) #Assuming Rabi frequency = 1. rad/microsec

mis_ref = nx_mis(G)
print(mis_ref)

In [ ]:
layers = 1
time_unit = 1000 # = 1 microsecond
detuning_amplitude = 1. # Must be between 0 and Rabi frequency, 1. is reasonable
guess = {'t': np.random.uniform(8, 10, layers),
         's': np.random.uniform(1, 3, layers)}

# Parametrized sequence
seq = Sequence(reg, Chadoq2)
seq.declare_channel('ch0','rydberg_global')

t_list = seq.declare_variable('t_list', size=layers)
s_list = seq.declare_variable('s_list', size=layers)

if layers == 1:
    t_list = [t_list]
    s_list = [s_list]

for t, s in zip(t_list, s_list):
    pulse_1 = Pulse.ConstantPulse(time_unit*t, 1., 0., 0)
    pulse_2 = Pulse.ConstantPulse(time_unit*s, 1., detuning_amplitude, 0)

    seq.add(pulse_1, 'ch0')
    seq.add(pulse_2, 'ch0')

# Bounds for max total pulse length (machine max = 100, reasonable values < 20)
max_time = 10.
step_time = max_time/layers
lb = []
ub = []
for t in t_list:
    lb.append(0.016) # minimum length of laser pulse
    ub.append(step_time*0.7)
for s in s_list:
    lb.append(0.016)
    ub.append(step_time*0.3)
np_ub = np.array(ub)
np_lb = np.array(lb)

In [ ]:
distances = pdist(pos)
Q = Chadoq2.interaction_coeff / pdist(pos) ** 6
Omega = np.average(Q)
delta_0 = -5  # just has to be negative
delta_f = -delta_0  # just has to be positive
T = 4000  # time in ns, we choose a time long enough to ensure the propagation of information in the system

In [ ]:
adiabatic_pulse = Pulse(
    InterpolatedWaveform(T, [1e-9, Omega, 1e-9]),
    InterpolatedWaveform(T, [delta_0, 0, delta_f]),
    0,
)
seq = Sequence(reg, Chadoq2)
seq.declare_channel("ising", "rydberg_global")
seq.add(adiabatic_pulse, "ising")
seq.draw()

In [ ]:
simul = Simulation(seq)
results = simul.run()
final = results.get_final_state()
count_dict = results.sample_final_state()

In [ ]:
plot_distribution(count_dict, mis_ref)

In [ ]:
cmap = ['royalblue','coral']
clist = [cmap[int(c)] for c in get_mis_string(count_dict)]
f = plt.figure()
nx.draw(G, pos=pos, node_color=clist, with_labels=True, ax=f.add_subplot(111))